### Train and save prediction model

In [1]:
import pandas as pd
dfList = []
for i in range(1,32):
    print(".",end='', flush=True)
    dfList.append(pd.read_json("./data/return-data/2017-01-{0:02d}.txt".format(i), lines=True))
print("done!")
df = pd.concat(dfList)

...............................done!


In [2]:
df['c_0'] = df.basket.map(lambda x: x.count(0))
df['c_1'] = df.basket.map(lambda x: x.count(1))
df['c_2'] = df.basket.map(lambda x: x.count(2))
df['c_3'] = df.basket.map(lambda x: x.count(3))
df['c_4'] = df.basket.map(lambda x: x.count(4))

In [3]:
# make sure all possible values are defined
df["zipCode"] = df["zipCode"].astype('category',categories=[i for i in list(range(1000,10000))])

/Users/pbaier/anaconda3/envs/analyse/lib/python3.5/site-packages/ipykernel_launcher.py:2: FutureWarning: specifying 'categories' or 'ordered' in .astype() is deprecated; pass a CategoricalDtype instead
  


In [4]:
dummies = pd.get_dummies(df.zipCode)
df2 = pd.concat([df, dummies], axis=1)
df3 = df2.drop(["basket", "zipCode", "transactionId"], axis=1)

In [5]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(df3, test_size=0.3, random_state=0)

In [6]:
X_train = train[train.columns[1:]]
y_train = train["returnLabel"]
X_test = test[test.columns[1:]]
y_test = test["returnLabel"]

In [7]:
from sklearn.linear_model import LogisticRegression
logreg = LogisticRegression()
logreg.fit(X_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [8]:
test_scores_reg = logreg.predict_proba(X_test)
from sklearn import metrics
fpr_reg, tpr_reg, _ =  metrics.roc_curve(y_test, test_scores_reg[:,1])
metrics.auc(fpr_reg, tpr_reg)

0.852830915062944

In [9]:
from sklearn.externals import joblib
joblib.dump(logreg, "model/model.pkl")

['model/model.pkl']

### Learn and save imputer

In [10]:
X_train.head()

,totalAmount,c_0,c_1,c_2,c_3,c_4,1000,1001,1002,1003,...,9990,9991,9992,9993,9994,9995,9996,9997,9998,9999
281,148,2,0,0,2,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
57,708,3,1,2,3,2,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
273,744,2,1,1,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
725,160,1,0,1,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
55,304,1,2,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [11]:
imputer = {}

In [12]:
imputer["totalAmount"] = X_train["totalAmount"].mean()

In [13]:
imputer["zipCode"] = df["zipCode"].value_counts().index[0]

In [14]:
def genBasket(cat):
    return [cat for i in range (0, (int)(X_train["c_"+str(cat)].median()))]

basket = []
for i in range(0,5):
    basket = basket + genBasket(i)

imputer["basket"] = basket

In [15]:
joblib.dump(imputer, "model/imputer.pkl")

['model/imputer.pkl']